CS591 Project 1
<br></br>
Peter Gilbert
<br></br>
<b>Problem 1:</b> BNF NOTATION
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\textit{program} & ::= & \textit{statement} \\
                 &  |  & \textit{statement} \ \textit{program} \\
 & & \\
\textit{statement} & ::= & \textbf{function} \ \textit{identifier, arguments, statement} \\
                 &  |  & \textbf{return} \ \textit{value} \\
                 &  |  & \textbf{assign} \ \textit{targets,value} \\
 & & \\
\textit{expression} & ::= & \textbf{BinOp} \ \textit{left, right, operator} \\
                 &  |  & \textbf{Call} \ \textit{func, arguments} \\
                 &  |  & \textbf{Num} \ \textit{object} \\
                 &  |  & \textbf{Name} \ \textit{identifier} \\
 & & \\
\textit{operator} & ::= & \textbf{Add} \\
                 &  |  & \textbf{Sub} \\
                 &  |  & \textbf{Mult} \\
                 &  |  & \textbf{Div} \\
\end{eqnarray*}

## Problem 2: checking syntax

In [66]:
#evaluating and checking the different parts of the ast
#checks if the program will conform to the syntax of the language
#part 2
import ast
import inspect

class evaluate(ast.NodeVisitor):
    
    def __init__(self):
        self.env = {}
        
    def visit_Module(self,node):
        results = [self.visit(s) for s in node.body]
        return all(results)
    
    def visit_FunctionDef(self,node):
        #for x in node.args.args:
        #    self.env[x.arg] = True
        results = [self.visit(s) for s in node.body] #checks body
        #argresults = [self.visit(x) for x in node.args.args] #checks args
        return all(results) #checks if everything in the lists are true
    
    def visit_Return(self,node):
        return self.visit(node.value)
    
    def visit_BinOp(self,node):
        node.left.env = self.env
        node.right.env = self.env
        if(type(node.op) is (ast.Add or ast.Sub or ast.Mult or ast.Div)):
            return self.visit(node.left) and self.visit(node.right)
        else:
            return False
        
    def visit_Num(self, node):
        return True
    
    def visit_Expr(self, node):
        return self.visit(node.value)
    
    def visit_Assign(self,node):
        #env = node.env
        #node.value.env = env
        value = self.visit(node.value)
        #env[node.targets[0].id] = value
        return value
    
    def visit_Name(self,node):
        return True
    
    def visit_Call(self,node):
        return self.visit(node.func)
    
    def visit_If(self,node):
        return self.visit(node.test)
    
def g(x):
    return 2+x
b = inspect.getsource(g)
print(ast.parse(b))
print(ast.dump(ast.parse(b)))
print()
print(evaluate().visit(ast.parse(b)))

Module(body=[FunctionDef(name='g', args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Num(n=2), op=Add(), right=Name(id='x', ctx=Load())))], decorator_list=[], returns=None)])

True


## Problem 3

In [67]:
import pendulum
from pendulum import DateTime
from datetime import datetime
import timeit
import astor

#pendulum is a datetime library similar to arrow
#https://github.com/sdispater/pendulum
#it allows more accurate datetime and allows timezone support
#as well as easy conversion between strings and timezones
#so I'm going to check it's speed at checking the current time and compare it to the python datetime library
#and see if it's faster then change all the instances of datetime to DateTime
#I found that finding the current time by just overriding the datetime function had pretty similar times 
#so I tried parsing and found that pendulum's parse method is much quicker

print(datetime.now(), "here")
print(DateTime.now(), "here")
print(DateTime.now('Japan'), "in Japan")

timesetup = '''
from pendulum import DateTime
from datetime import datetime
import pendulum'''

pendversion = '''
def pendTest():
    DateTime.now()'''

pythonVersion = '''
def pythonTest():
    datetime.now()'''

pendversion2 = '''
date_time_str = '2018-06-29 08:15:27.243860'
date_time_obj = DateTime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')'''

pendversion3 = '''
pendulum.parse('2018-06-29 08:15:27.243860')
'''

pythonVersion2 = '''
date_time_str = '2018-06-29 08:15:27.243860'
date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')'''

#DateTime tests
#testing datetime vs DateTime using the now function which sees the current time
print ((timeit.timeit(setup = timesetup, stmt = pendversion, number = 1000000)), "for now with pendulum")
print ((timeit.timeit(setup = timesetup, stmt = pythonVersion, number = 1000000)), "for now with python")

#DateTime tests using parse
print ((timeit.timeit(setup = timesetup, stmt = pendversion2, number = 100000)),"for parsing with pendulum")
print ((timeit.timeit(setup = timesetup, stmt = pendversion3, number = 100000)),"for parsing with pendulum version 3")
print ((timeit.timeit(setup = timesetup, stmt = pythonVersion2, number = 100000)), "for parsing with python datetime")

#pendulum's own parse method seems much quicker than the strptime method from datetime, so let's change it so that whenever
#strptime is called that we instead use the pendulum parse method
#date_time_str = '2018-06-29 08:15:27.243860'
#date_time_obj = DateTime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
#b = inspect.getsource(datetime.strptime)
#print(ast.dump(ast.parse(b)))
#built in method so i can't check it
def f():
    date_time_str = '2018-06-29 08:15:27.243860'
    datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    
def g():
    date_time_str = '2018-06-29 08:15:27.243860'
    pendulum.parse(date_time_str)

print()

fast = inspect.getsource(f)
gast = inspect.getsource(g)

print(ast.dump(ast.parse(fast)))
print()
print(ast.dump(ast.parse(gast)))
print()

class changeParser(ast.NodeTransformer):
    #change call's name, attribute's name, and take just the first parameter
    def visit_Call(self,node):
        x = []
        if(node.func.attr == 'strptime'):
            x = node.args[0]
            node.args = [x]
            a =\
              ast.copy_location(
                ast.Call(
                    func = ast.Attribute(
                        value = ast.Name(id='pendulum',ctx=ast.Load()),
                        attr='parse',
                        ctx=ast.Load()
                    ),
                    args=[x],
                    keywords=[]
                ),
                node
              )
            return a
        else:
            return node
        
print(astor.to_source(ast.parse(fast)))
x = changeParser().visit(ast.parse(fast))
print(astor.to_source(x))
print(evaluate().visit(x))
#we successfully changed it from the datetime.strptime function into the pendulum parse function
#so now we can make it so whenever that function is called, we can use the pendulum function instead
#which is much faster.

2019-09-27 23:41:53.676441 here
2019-09-27T23:41:53.676441-04:00 here
2019-09-28T12:41:53.676441+09:00 in Japan
0.07283047396595066 for now with pendulum
0.07461154001066461 for now with python
1.6931661507878744 for parsing with pendulum
0.7565588435882091 for parsing with pendulum version 3
1.103425951589088 for parsing with python datetime

Module(body=[FunctionDef(name='f', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='date_time_str', ctx=Store())], value=Str(s='2018-06-29 08:15:27.243860')), Expr(value=Call(func=Attribute(value=Name(id='datetime', ctx=Load()), attr='strptime', ctx=Load()), args=[Name(id='date_time_str', ctx=Load()), Str(s='%Y-%m-%d %H:%M:%S.%f')], keywords=[]))], decorator_list=[], returns=None)])

Module(body=[FunctionDef(name='g', args=arguments(args=[], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Assign(targets=[Name(id='date_time_str', ctx=Store())], 